In [1]:
import pandas as pd
import numpy as np

# Import libraries for tools
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Model Training 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.ensemble import RandomForestClassifier

In [3]:
data = pd.read_csv(r'F:\Machine_learning\Credit_Card_Default_Prediction_and_Model_Deployment_Project\notebooks\data\UCI_Credit_Card.csv',index_col="ID")
data_train,data_test = train_test_split(data,random_state=42,test_size=.15)
df= data.copy()
data_train.sample(10).T


ID,27494,27263,17441,22134,20092,13989,23838,16916,23027,13997
LIMIT_BAL,160000.0,50000.0,30000.0,80000.0,100000.0,50000.0,360000.0,390000.0,610000.0,340000.0
SEX,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0
EDUCATION,3.0,2.0,2.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0
MARRIAGE,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0
AGE,51.0,38.0,32.0,22.0,42.0,38.0,54.0,64.0,43.0,29.0
PAY_0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
PAY_2,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
PAY_3,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0
PAY_4,-2.0,2.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
PAY_5,-1.0,2.0,-2.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
lower_bound = df['column_name'].quantile(0.025)  # 2.5th percentile
upper_bound = df['column_name'].quantile(0.975)